In [1]:
import pandas as pd
import numpy as np
import os

# 确定 Xsens Dot 和关节位置对应关系

In [26]:
# Xsens Dot 编号和贴放部位的对应关系
dots_number = [0,1,2,3,4]
dots_serial = ["097","086","094","076","090"]
bodies = ["pelvis","tibia_r","femur_r","tibia_l",'femur_l']

bodies_dots = {"pelvis":[0,dots_serial[0]],
               "tibia_r":[4,dots_serial[4]],
               "femur_r":[2,dots_serial[2]],
               "tibia_l":[3,dots_serial[3]],
               "femur_l":[1,dots_serial[1]]}

# 合并多个 Xsens Dot 数据为一个文件

In [80]:
root_path = "/Users/wangshuaibo/Library/CloudStorage/OneDrive-bsu.edu.cn/Archive/代码_OpenSense/OpenSense_Python/OpenSenseExample/OpenSenseExample_withoutTorsoFoot_Dot_20230525"
os.chdir(root_path)
os.getcwd()

'/Users/wangshuaibo/Library/CloudStorage/OneDrive-bsu.edu.cn/Archive/代码_OpenSense/OpenSense_Python/OpenSenseExample/OpenSenseExample_withoutTorsoFoot_Dot_20230525'

In [81]:
xsens_dot_path = "/Users/wangshuaibo/Library/CloudStorage/OneDrive-bsu.edu.cn/Archive/代码_OpenSense/OpenSense_Python/OpenSenseExample/OpenSenseExample_withoutTorso_20220525/XsensDot"
xsens_dot_name = "20230525_102929"
frame_rate = 60
quats_dfs = pd.DataFrame()
for body in bodies:
    filename = f"Xsens DOT{int(bodies_dots[body][0])}_{xsens_dot_name}_{bodies_dots[body][1]}.csv"
    file_path = os.path.join(xsens_dot_path,filename)
    df = pd.read_csv(file_path, skiprows=10)
    quats = ['Quat_W', 'Quat_X', 'Quat_Y', 'Quat_Z']
    new_quats = [body + "_" + quat for quat in quats]
    df[new_quats] = df[quats]
    quats_dfs = pd.concat((quats_dfs,df[new_quats]),axis=1)
    quats_dfs = quats_dfs.round(17)
time = np.arange(0, len(quats_dfs)*1/frame_rate, 1/frame_rate)
quats_dfs.insert(0,"time",time)

In [82]:
# 由于数据传输问题，dataframe 末尾几行会出现 Nan
# 后续数据处理会报错
nan_rows = quats_dfs[quats_dfs.isna().any(axis=1)]
# 打印包含 Nan 的行数
print("总行数为：\n",len(quats_dfs))
print("包含 Nan 的行：\n",nan_rows.index)
quats_dfs.dropna(axis=0, inplace=True)
print("已经删除包含 NaN 的行")

# 保存数据
quats_dfs.to_csv("XsensMVNOritention.csv",index=False)

总行数为：
 346
包含 Nan 的行：
 Int64Index([342, 343, 344, 345], dtype='int64')
已经删除包含 NaN 的行


# 转换四元数为Opensim 格式

In [83]:
path = "/Users/wangshuaibo/Library/CloudStorage/OneDrive-bsu.edu.cn/Archive/代码_OpenSense/OpenSense_Python/OpenSenseExample/OpenSenseExample_withoutTorsoFoot_Dot_20230525"

In [84]:
# 设置表头
bodies_imu = [body+"_imu" for body in bodies]
print(bodies_imu)
opensim_sto_columns = "time\t"
for body_imu in bodies_imu:
    opensim_sto_columns = opensim_sto_columns + body_imu + "\t"

opensim_sto_columns = opensim_sto_columns.strip() + "\n"

['pelvis_imu', 'tibia_r_imu', 'femur_r_imu', 'tibia_l_imu', 'femur_l_imu']


In [85]:
frame_rate = 60
meatdata = f"""DataRate={frame_rate:.6f}
DataType=Quaternion
version=3
OpenSimVersion=4.4-2022-10-11-798caa8
endheader
"""
header = meatdata + opensim_sto_columns
header

'DataRate=60.000000\nDataType=Quaternion\nversion=3\nOpenSimVersion=4.4-2022-10-11-798caa8\nendheader\ntime\tpelvis_imu\ttibia_r_imu\tfemur_r_imu\ttibia_l_imu\tfemur_l_imu\n'

In [86]:
old_filename = "XsensMVNOritention.csv"
oldfile_path = os.path.join(path,old_filename)

new_filename = "XsensMVNOritention.sto"
newfile_path = os.path.join(path, new_filename)

In [87]:
# 先写入表头内容
with open(newfile_path,"w") as new_f:
    new_f.write(header)

In [88]:
# 写入四元数数据
with open(oldfile_path, "r") as old_f, open(newfile_path, "a") as new_f:
    line_index = 0
    for line in old_f:
        # 去除第一行的表头不进行记录
        line_index += 1
        if line_index > 1:
            new_line = line.split(",")
            # print(new_line)
            str_new = ""
            index = 0
            for i in new_line:
                if index % 4 == 0:
                    str_new = str_new + i.strip() + "\t"
                else:
                    str_new = str_new + i.strip() + ","
                index = index + 1
            new_f.write(str_new.strip()+"\n")